# MAD Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
22/03/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from scipy.optimize import least_squares

from scipy import ndimage
import os
import scipy.io as sio
from scipy.ndimage import rotate
from skimage.io import imread


import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.linalg import lstsq


In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x00000204C65AA380>
Using matplotlib backend: TkAgg


# Traitement images

In [3]:
# FONCTION DE ROTATION D'IMAGE 
# entrée : une image
# ginput : bord gauche et bord droit du bec
# sortie : angle de rotation, image J après rotation, coordonées des points selectionnés
def im_rotate(image):
    plt.imshow(image, cmap='gray')
    plt.title("Cliquez sur les bords du bec (gauche puis droit)")

    # récuperer les coordonnées des bords du bec
    bec_coord = plt.ginput(2)
    plt.close()
    # print(f"Cordonnées des bords du bec: {np.round(bec_coord, 2)}")
        
    # Calculer l'angle pour aligner l'image
    (x1, y1), (x2, y2) = bec_coord
    rot_angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    # print(f"Angle à corriger: {rot_angle} degrés")

    # Calculer le centre de l'image
    (h, w) = image.shape
    center = (w // 2, h // 2)

    # Créer une matrice de rotation centrale
    rot_mat = cv2.getRotationMatrix2D(center, rot_angle, 1.0)

    # Appliquer la rotation
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)

    # plt.imshow(img_rot, cmap='gray')
    # plt.title("Image alignée")
    
    return rot_mat, img_rot, bec_coord

# ROGNER IMAGE
# entrée : image, coordonées du bec (_,_, xy_bec = im_rotate(J))
def rogner(image, xy_bec):
    # img_rogn = np.flipud(image)
    x_BG = xy_bec[0][0]
    y_BG = xy_bec[0][1] 
    x_BD = xy_bec[1][0]
    y_BD = xy_bec[1][1]
    
        # bornes 
    wmin = int(x_BG)-20     # width min
    wmax = int(x_BD)+20
    hmin = int(y_BG-42)     # height min
    hmax = int(y_BG+100)
    img_rogn = image[hmin:hmax, wmin:wmax] 
    
    return img_rogn

def bord_anche(image):
    points = []
    I = np.flipud(image)
    (H, W) = I.shape
    # print(H,W)
    plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    plt.axis('equal')
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.title("Cliquez sur les bords de l'anche")
    anche_coord = plt.ginput(2)
    plt.close()
    return anche_coord

# CALCUL DU CANAL D'nbr_anche
# entrée :  image, sensibilite de la caméra en mm/pix, coordonnées des bords de l'anche,
def surface_canal(image, S, seuil, anche_coord):
    # s'assurer que I est bien en binaire
    _, I = cv2.threshold(image, seuil, 255, cv2.THRESH_BINARY)

    (x1, y1), (x2, y2) = map(lambda p: (int(p[0]), int(p[1])), anche_coord)
    # initialisation matrices vides
    Xv = np.arange(x1, x2, dtype=int)   # vecteur de la taille de la largeur de l'image coupée
    Yv_m = []
    Yv_r = []
    

# PROBLEME POUR GENERALISER A UNE TRANCHE ENTIERE
    for xv in Xv:
        Iv = I[:, xv].astype(float)   # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
        Iv_2 = np.round(np.gradient(Iv)).astype(int)     # Dérivée centrée (flottante) + Arrondir + convertir en entiers
       
        # seuillage trache par tranche
        smin = -30
        smax = -smin
        pos1 = np.where(Iv_2 < smin)[0][0] if any(Iv_2 < smin) else None
        pos2 = np.where(Iv_2 > smax)[0][0] if any(Iv_2 > smax) else None
        (H, W) = I.shape
        Y = np.arange(1,H+1)
    
        if pos2 is not None:
            # Yv_m[i] = Y[pos2]
            Yv_m.append(Y[pos2])
        else:
            # Yv_m[i] = np.nan
            Yv_m.append(0)
            # print('error!!!')  # Peu probable parce que détectera toujours la frontière lèvre/anche

        if pos1 is not None:
            # Yv_r[i] = Y[pos1]
            Yv_r.append(Y[pos1])
        else:
            # Yv_r[i] = Yv_m[i]  # Lorsque le canal d'anche est fermé, il n'y aura pas de Yv_r. On lui donne la même valeur que Yv_m comme ça la différence sera de 0.
            Yv_r.append(Yv_m)
    
    # for m, r in zip(Yv_m, Yv_r):
    #     print(f"Type de m: {type(m)}, Type de r: {type(r)}")
           
    # Calculer la différence entre les deux courbes
    diff = [0 if m == 0 or r == 0 else (int(m) - int(r)) for m, r in zip(Yv_m, Yv_r)]
    diff = np.abs(diff)
    # les Nan sont remplacés par des 0
    # diff = np.nan_to_num(diff)

    # Calculer l'aire en utilisant la méthode des trapèzes en pixel²
    area = np.trapz(diff, Xv)

    # mise en mm²
    surf_pix = S ** 2  # surface d'un pixel
    dim_canal = area / surf_pix  # surface en mm² du canal
        
    return Xv, Yv_r, Yv_m, dim_canal



# Application

In [7]:
# SENSIBILITE CAPTEUR
S_force =  0.04026 # mV/N
G_force = 100
S_cam = 83.35
gain_acqui = 900

# cannal d'acquisition
Cannal = 1
dossier = 'mesures_2024.09.26'
nom_anche = '/anche'
nbr_anche = 5
seuil = 150

In [8]:
# J = cv2.imread(dossier + nom_anche + '1_0.bmp', cv2.IMREAD_GRAYSCALE)
# _, J = cv2.threshold(J, seuil, 255, cv2.THRESH_BINARY)
# J = J.astype(np.float32)
# plt.imshow(J, cmap='gray')

In [9]:
plt.close('all')

# Initialisation sur une image

# Lecture de l'image en nuance de gris, mise en noir et blanc
# J = cv2.imread(dossier + nom_anche + '1_0.bmp', cv2.IMREAD_GRAYSCALE)
J = cv2.imread(dossier + '/image_1_0.jpg', cv2.IMREAD_GRAYSCALE)
# print(dossier + '/image_1_0.jpg')
_, J = cv2.threshold(J, seuil, 255, cv2.THRESH_BINARY)
(h, w) = J.shape

rot_mat, J_rot,  xy_bec = im_rotate(J)
I = rogner(J_rot, xy_bec)
anche_coord = bord_anche(I)
# print(anche_coord[1][1])

# Import data
for i in range(nbr_anche):
    # with open(dossier + f'/air_canal_A{i+1}.txt', 'w') as file:
    force = np.loadtxt(dossier + f'/data_anche{i+1}.txt')
    force = force[:,2]
    force = ((force-min(force))*10**-3)/(S_force*gain_acqui/100)
    # print(force-min(force))
    with open(dossier + f'/Anche{i+1}.txt', 'w') as file:
        dim_c = []
        plt.figure()
        for f in range(30):
            # img_path = f'mes_2024.03.22/A{i+1}_{f}.bmp'
            img_path = dossier + f'/image_{i+1}_{f}.jpg'
            if not os.path.exists(img_path):
                # print(f"File {img_path} does not exist. Skipping to next iteration.")
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)
            img_rot = cv2.warpAffine(img, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
            img =  rogner(img_rot, xy_bec)
            # img = np.flipud(img)
            Xv, Yv_r, Yv_m, dim_canal = surface_canal(img, S_cam , seuil, anche_coord)
            dim_c.append(dim_canal)
            plot_fig = True
            if plot_fig :
                # plt.figure()
                plt.imshow(img, cmap='gray')
                plt.plot(Xv, Yv_r, 'm.-', linewidth=0.1)
                plt.plot(Xv, Yv_m, '.-', linewidth=0.1)
                filtered_Xv = [x for x, y in zip(Xv, Yv_m) if y != 0]
                filtered_Yv_m = [y for y in Yv_m if y != 0]
                # plt.plot(filtered_Xv, filtered_Yv_m, '.-', linewidth=0.1)
                # print(f"L'aire entre les deux courbes est : {np.round(dim_canal, 2)} mm²")
            file.write(f'Image_{i+1}_{f}, {force[f]}, {seuil}, {dim_canal} \n')
            # print(force)
            # file.write(str(dim_canal) + '\n')

# ATTENTION : si un fichier s'appelle déjà de cette façon, python va écrire à la suite, ce qui occasionne des problèmes de traille de datas pour le traitement plus tard

In [10]:
S_force = 0.04026/100 
gain_acqui = 900 
plt.close('all')
plt.figure(figsize=(10,6))

for i in range(nbr_anche):
    # data = np.genfromtxt(f'mes_2024.09.10/anche{i+1}/Anche{i+1}.txt', delimiter=',')
    data = np.genfromtxt(dossier + f'/Anche{i+1}.txt', delimiter=',')
    force = data[:,1]     
    surf_canal = data[:,3]
    
    plt.plot(force, surf_canal, '*-', linewidth=1.8, label=f'anche{i+1}')
    # identification_modele(F, surf_c)
    plt.xlabel('Force (N)')
    plt.ylabel('Surface (mm²)')
    plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
    plt.legend()
    plt.grid(True)
    # plt.axis([-30, 350, -2, 13])
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

# OBSOLETE 

In [10]:
# Comparaison avec le traitement d'image de octave 
#### ATTENTION : supprimer à la mains le document portant le même nom car python écrit à la suite, ce qui créer des problème de taille de vecteur
comparaison_pyth_oct = False

if comparaison_pyth_oct:
    plt.close('all')
    # plt.figure(figsize=(10,6))

    for i in range(nbr_anche):
        plt.figure()
        force = np.loadtxt(dossier + f'/force_A{i+1}.txt')
        force = force/S_force
        surface_canal = np.loadtxt(dossier + f'/air_canal_A{i+1}.txt')
        # surface_canal = surface_canal/S_force
        surf_c_octave = np.loadtxt(dossier + f'/surf_c_A{i+1}.txt')
        
        # Erreur absolue moyenne (MAE)
        mae = np.mean(np.abs(surface_canal - surf_c_octave))
        print(f"Erreur absolue moyenne (MAE) : {mae}")

        # Eerreur quadratique moyenne (MSE)
        mse = np.mean((surface_canal - surf_c_octave)**2)
        print(f"Erreur quadratique moyenne (MSE) : {mse}")

        # surf_c_octave = surf_c_octave/S_force
        plt.plot(force, surface_canal, '*-', linewidth=1.8, label=f'Python')
        plt.plot(force, surf_c_octave, '*-', linewidth=1.8, label=f'Octave')
        # identification_modele(F, surf_c)
        plt.xlabel('Force')
        plt.ylabel('Surface (mm²)')
        plt.title('Surface du canal d\'anche en fonction de la force \n d\'appui de la lèvre articielle')
        plt.legend()
        
        message = f'erreur absolue moyenne (MAE) : {np.round(mae,3)} \nerreur quadratique moyenne (MSE) : {np.round(mse,3)}'
        props = dict(boxstyle='round,pad=0.5', facecolor='white', edgecolor='lightgray')
        plt.text(-0.5, -0.1, message, fontsize=10,
        verticalalignment='bottom', horizontalalignment='left', bbox=props)
        plt.grid(True)
        # plt.axis([-30, 350, -2, 13])
        plt.xticks(fontsize=12)
        plt.yticks(fontsize=12)
        plt.show()

Traitement données octave

In [4]:
# for i in range(nbr_anche):
#     # Metre le fichier . mat en .txt        
#     # Lecture du fichier MATLAB (.m) et écriture dans un fichier texte (.txt)
#     with open(dossier + f'/A{i+1}.mat', 'r') as file:
#         matlab_code = file.read()
        
#     # Écriture du contenu dans un fichier texte
#     with open(dossier + f'/A{i+1}.txt', 'w') as file:
#         file.write(matlab_code)
        
#     # Ouvrir le fichier d'entrée en mode lecture
#     with open(dossier + f'/A{i+1}.txt', 'r') as f:
#         lines = f.readlines()

#     # Initialiser une variable pour suivre le numéro de fichier en cours
#     current_file_index = 1

#     # Parcourir les lignes du fichier
#     for line in lines:
#         if line.startswith('#'):
#             # Si une ligne commence par '#', cela indique un nouveau fichier
#             current_file_index += 1
#         else:
#             # Écrire le contenu de la ligne dans le fichier correspondant
#             with open(dossier + f'/matrix{current_file_index}_A{i+1}.txt', 'a') as current_file:
#                 current_file.write(line)
    
#     # def_r 
#     with open(dossier + f'/matrix6_A{i+1}.txt', 'r') as f:
#         lines = f.readlines()

#     # Initialiser une variable pour suivre le numéro de fichier en cours
#     current_file_index = 1
#     # Parcourir les lignes du fichier
#     for line in lines:
#         # Diviser la ligne en valeurs individuelles en utilisant l'espace comme délimiteur
#         values = line.split()
#         values = np.array(values)
#         # Écrire les valeurs dans le fichier correspondant
#         with open(dossier + f'/def_r_A{i+1}.txt', 'a') as current_file:
#             for value in values:
#                 current_file.write(value + ' ')
#             current_file.write('\n')

#     # der_m
#     with open(dossier + f'/matrix10_A{i+1}.txt', 'r') as f:
#         lines = f.readlines()
#     current_file_index = 1
#     for line in lines:
#         values = line.split()
#         values = np.array(values)
#         with open(dossier + f'/def_m_A{i+1}.txt', 'a') as current_file:
#             for value in values:
#                 current_file.write(value + ' ')
#             current_file.write('\n')
            
#     # surf_c
#     with open(dossier + f'/matrix14_A{i+1}.txt', 'r') as f:
#         lines = f.readlines()
#     current_file_index = 1
#     for line in lines:
#         values = line.split()
#         values = np.array(values)
#         with open(dossier + f'/surf_c_A{i+1}.txt', 'a') as current_file:
#             for value in values:
#                 current_file.write(value + ' ')
#             current_file.write('\n')

In [5]:
# # Metre le fichier . mat en .txt        
# # Lecture du fichier MATLAB (.m) et écriture dans un fichier texte (.txt)
# with open(dossier + '/A1.mat', 'r') as file:
#     matlab_code = file.read()
    
# # Écriture du contenu dans un fichier texte
# with open(dossier + '/A1.txt', 'w') as file:
#     file.write(matlab_code)


In [6]:
# # Ouvrir le fichier d'entrée en mode lecture
# with open(dossier + '/A1.txt', 'r') as f:
#     lines = f.readlines()

# # Initialiser une variable pour suivre le numéro de fichier en cours
# current_file_index = 1

# # Parcourir les lignes du fichier
# for line in lines:
#     if line.startswith('#'):
#         # Si une ligne commence par '#', cela indique un nouveau fichier
#         current_file_index += 1
#     else:
#         # Écrire le contenu de la ligne dans le fichier correspondant
#         with open(dossier + f'/matrix{current_file_index}.txt', 'a') as current_file:
#             current_file.write(line)

In [7]:
# # def_r 
# with open(dossier + '/matrix6.txt', 'r') as f:
#     lines = f.readlines()

# # Initialiser une variable pour suivre le numéro de fichier en cours
# current_file_index = 1
# # Parcourir les lignes du fichier
# for line in lines:
#     # Diviser la ligne en valeurs individuelles en utilisant l'espace comme délimiteur
#     values = line.split()
#     values = np.array(values)
#     # Écrire les valeurs dans le fichier correspondant
#     with open(dossier + '/def_r.txt', 'a') as current_file:
#         for value in values:
#             current_file.write(value + ' ')
#         current_file.write('\n')

# # der_m
# with open(dossier + '/matrix10.txt', 'r') as f:
#     lines = f.readlines()
# current_file_index = 1
# for line in lines:
#     values = line.split()
#     values = np.array(values)
#     with open(dossier + '/def_m.txt', 'a') as current_file:
#         for value in values:
#             current_file.write(value + ' ')
#         current_file.write('\n')
        
# # surf_c
# with open(dossier + '/matrix14.txt', 'r') as f:
#     lines = f.readlines()
# current_file_index = 1
# for line in lines:
#     values = line.split()
#     values = np.array(values)
#     with open(dossier + '/surf_c.txt', 'a') as current_file:
#         for value in values:
#             current_file.write(value + ' ')
#         current_file.write('\n')